# Argument Linking of GLUCOSE Rules

In [1]:
import json
import pickle
import os
import amrlib
import spacy
import re
import penman

# setup spacy extension for amrlib
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')

# setup faa_aligner
os.environ["FABIN_DIR"] = "/home/nadia/fast_align/build"
from amrlib.alignments.faa_aligner import FAA_Aligner
inference = FAA_Aligner()

Load GLUCOSE data into memory

In [2]:
with open('./glucose_data/story_data_final.json','r') as f:
        stories = json.load(f)
print('loaded ',len(stories), ' stories from "./glucose_data/story_data_final.json"')

def load_graphs(path):
    graphs = []
    for file in os.listdir(path):
        if file.endswith('.pkl'):
            with open(path+file, 'rb') as f:
                graphs.append(pickle.load(f))
    return(graphs)

story_graphs = load_graphs('./story_graphs/')
print('loaded ', len(story_graphs), ' story graphs into memory')

loaded  4584  stories from "./glucose_data/story_data_final.json"
loaded  4488  story graphs into memory


### AMR-String to Graph-Triples



In [3]:
def get_graph_triples(amr_parse):

    lines = []
    amr_parse = amr_parse[:-1]
    for line in amr_parse.split('\n'):
        if line.startswith('#'):
            continue
        else:
            lines.append(line.strip())
        
    current_nodes = [lines[0].replace('(','')]
    graph_triples = []
    for line in lines[1:]:
        out_node = current_nodes[-1]
        relation = line.split(' ')[0]
        line = line.replace(relation+' ','')
        node = line.strip().replace('(','').replace(')','')
        graph_triples.append((out_node, relation, node))
        depth = line.count(')')-line.count('(')
        
        if line.endswith(')'):
            if depth > 0:
                current_nodes = current_nodes[:len(current_nodes)-(depth)]
        elif '/' not in line:
            continue
        else:
            current_nodes.append(node)
    return(graph_triples)

### Gap-Fillers

In [4]:
replacers = {'someone_a':'Aria', "someone_a'":"Aria'", "someone_a's":"Aria's", 'someone_b':'Deneris',
             "someone_b's":"Deneris'", 'someone_c':'Spook', "someone_c's":"Spook's", 'someone_d':'Kirk',
             'somepeople_a':'Aria', "somepeople_a's":"Aria's", 'somepeople_b':'Deneris', 
             'something_a':'crown', "something_a's":"crown's", 'something_b':'throne', "something_b's":"throne's", 
             'something_bs':"throne's",
             'somethings_a':'somethings',
             'somewhere_a':'somewhere'}

def rename_fillers(rule):
    
    rule = rule.split()
    for i, word in enumerate(rule):
        if word.replace(')','') in replacers.keys():
            rule[i] = replacers[word.replace(')','')]
    rule = ' '.join(rule)
    return(rule)

### Story Example

In [5]:
story_graph = story_graphs[0]
story = story_graph.nodes['1303_S']['story']
print('STORY:\n')
print(story)
rules = story_graph.edges[('1303_0', '1303_3')]['annotations'][0]['rules']
print('\n\nRULE EXAMPLE SPECIFIC:\n')
print(' - '.join(rules[0]),'\n')
print('\n\nRULE EXAMPLE GENERAL:\n')
print(' - '.join(rules[4]),'\n')

parses = [nlp(rules[0][0])._.to_amr(), nlp(rules[0][2])._.to_amr(), nlp(rules[4][0])._.to_amr(), nlp(rules[4][2])._.to_amr()]

for parse in parses:
    for line in parse:
        print(line)
    print('\n')

STORY:

Kelly was in the mood for a fruity, tasty treat. It was hot out, so she popped some grapes in the freezer. When they froze, she snacked on them one by one. She even added a few to her soda to keep it cold! Kelly loved her frozen grape treats.


RULE EXAMPLE SPECIFIC:

Kelly was in the mood for a fruity, tasty treat - Results in - Kelly has added a few to her soda to keep it cold 



RULE EXAMPLE GENERAL:

Someone_A was in the mood for Something_A (frozen treats)  - Results in - Someone_A has added Something_A to their drink to keep it cold 

# ::snt Kelly was in the mood for a fruity, tasty treat
(m / mood
      :domain (p / person
            :name (n / name
                  :op1 "Kelly"))
      :purpose (t / treat
            :mod (f / fruity)
            :mod (t2 / tasty)))


# ::snt Kelly has added a few to her soda to keep it cold
(a / add-02
      :ARG0 (p / person
            :name (n / name
                  :op1 "Kelly"))
      :ARG1 (f / few)
      :ARG2 (s / soda
  

In [6]:
print(get_graph_triples('\n'.join(parses[1])))

[('a / add-02', ':ARG0', 'p / person'), ('p / person', ':name', 'n / name'), ('n / name', ':op1', '"Kelly"'), ('a / add-02', ':ARG1', 'f / few'), ('a / add-02', ':ARG2', 's / soda'), ('s / soda', ':poss', 'p'), ('a / add-02', ':purpose', 'k / keep-02'), ('k / keep-02', ':ARG0', 'p'), ('k / keep-02', ':ARG1', 'c / cold-01'), ('c / cold-01', ':ARG1', 's')]


### 1.) Map Arguments of General Rule

GENERAL RULE: 

    Someone_A was in the mood for Something_A ((frozen treats)) 
    
    - Results in - 
    
    Someone_A has added Something_A to their drink to keep it cold



#### After parsing the sentences to AMR we want to extract graph triples with the matching GENERALIZER:

- SOMEONE_A:

    ('m / mood', ':domain', 'p / person') = ('a / add-02', ':ARG0', 'p / person')


- SOMETHING_A:

    ('m / mood', ':mod', 's / something') = ('a / add-02', ':ARG1', 's / something')

In [7]:
def map_arguments(rule):
    
    argument_mappings = []
    left_over = []
    
    # get general rule
    rule = rule[4]
    part1, part2 = rule[0].lower(), rule[2].lower()
    #print(part1)
    #print(part2)

    # rename fillers
    filler1 = [replacers[word.replace(')','')] for word in part1.split(' ') if '_' in word]
    filler2 = [replacers[word.replace(')','')] for word in part2.split(' ') if '_' in word]
    filler = {el:[] for el in filler1 if el in filler2}
    part1, part2 = rename_fillers(part1), rename_fillers(part2)
    #print(part1)
    #print(part2)
    # save index of fillers in original sentence
    for fill in filler:
        filler[fill] = [part1.split().index(fill), part2.split().index(fill)]
    #print(filler)

    # parse to amr
    doc1, doc2 = nlp(part1), nlp(part2)
    parse_part1, parse_part2 = doc1._.to_amr(), doc2._.to_amr()
    triples_part1, triples_part2 = get_graph_triples(parse_part1[0]), get_graph_triples(parse_part2[0])

    #print(parse_part1)
    #print(parse_part2)

    for fill in filler:
        fill = filler[fill][0]
        fill = re.sub('[()]', '', doc1[fill:fill+1]._.to_amr()[0].split('/ ')[1].split('\n')[0])
        print(fill)
        for triple in triples_part1:
            #print(triple)
            triple1 = (None,None)
            if not '_of' in triple[1]:
                if fill in triple[2]:
                    triple1 = triple
                    break
            else:
                if fill in triple[0]:
                    triple1 = triple
                    break
        for triple in triples_part2:
            triple2 = (None,None)
            if not '-of' in triple[1]:
                if fill in triple[2]:
                    triple2 = triple
                    break
            else:
                if fill in triple[0]:
                    triple2 = triple
                    break
        if None not in triple1 and None not in triple2:
            mapping = (triple1,triple2)
            argument_mappings.append(mapping)
        else:
            left_over.append(rule)
            continue
    #print(list(set(argument_mappings)))
    #print('\n\n')
    return((list(set(argument_mappings)),left_over))

#### Example 1

In [8]:
argument_mapping, left_over = map_arguments(rules)
print('ARGUMENT MAPPINGS:\n')
for mapping in argument_mapping:
    print(mapping)

person
crown
ARGUMENT MAPPINGS:

(('m / mood', ':domain', 'p / person'), ('a / add-02', ':ARG0', 'p / person'))
(('m / mood', ':mod', 'c / crown'), ('a / add-02', ':ARG1', 'c / crown'))


#### Example 2

In [11]:
rules2 = story_graph.edges[('1303_0', '1303_3')]['annotations'][1]['rules']
print('\n\nRULE EXAMPLE SPECIFIC:\n')
print(' - '.join(rules2[0]),'\n')
print('\n\nRULE EXAMPLE GENERAL:\n')
print(' - '.join(rules2[2]),'\n')
argument_mapping2, left_over2 = map_arguments(rules2)
print('\n\nARGUMENT MAPPING:\n')
for mapping in argument_mapping2:
    print(mapping)



RULE EXAMPLE SPECIFIC:

Kelly want(s) a fruity, tasty treat - Motivates - Kelly popped some grapes in the freezer 



RULE EXAMPLE GENERAL:

Someone_A want(s) A fruity, tasty treat - Motivates - Someone_A popped some grapes in the freezer 



ARGUMENT MAPPING:

(('w / want-01', ':ARG0', 'p / person'), ('p / pop-01', ':ARG0', 'p2 / person'))
